<a href="https://colab.research.google.com/github/Ryzhikov114/Portfolio/blob/main/scoring_bank_debt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Исследование надёжности заёмщиков

## Описание проекта:
Заказчик — кредитный отдел банка. Нужно разобраться, какие факторы влияют на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

https://drive.google.com/file/d/1q8kXFmg8klXBNjXTYjuifJqouG0Ny4q4/view?usp=sharing

In [150]:
!gdown 1q8kXFmg8klXBNjXTYjuifJqouG0Ny4q4

Downloading...
From: https://drive.google.com/uc?id=1q8kXFmg8klXBNjXTYjuifJqouG0Ny4q4
To: /content/data.csv
100% 3.32M/3.32M [00:00<00:00, 171MB/s]


- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита



In [151]:
import pandas as pd
import numpy as np

In [152]:
df = pd.read_csv('/content/data.csv')

In [205]:
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,children_category,education_category,family_status_category,income_type_category
0,1,-8437.673028,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,have_children,university,in_wedding,have_job
1,1,-4024.803754,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,have_children,school,in_wedding,have_job
2,0,-5623.422610,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,no_children,school,in_wedding,have_job
3,3,-4124.747207,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,have_children,school,in_wedding,have_job
4,0,340266.072047,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,no_children,school,in_wedding,unemployed


In [154]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [155]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [156]:
df = df.apply(lambda x: x.str.lower() if x.dtype=='object' else x)

Для создания скоринга нам нужны булевы данные из нашей таблицы это могут быть следущие столбцы:
- children есть дети - 1, нет детей - 0 
- education - есть высшее - 1, нет высшего - 0 
- family_status - в браке - 1, не в браке - 0
- income_type - работает - 1, не работает - 0

In [184]:
def children_group(x):
  if x == 0:
    return 'no_children'
  else:
    return 'have_children'

In [185]:
df['children_category'] = df['children'].apply(children_group)

In [193]:
df.groupby('children_category')['debt'].agg(['count', 'mean'])

,count,mean
children_category,,
have_children,7376,0.091920
no_children,14149,0.075129


Клиенты у которых нет детей, возвращают кредит хуже

In [159]:
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [187]:
def education_group(x):
  if x == 'высшее' or x == 'ученая степень':
    return 'university'
  else:
    return 'school'

In [188]:
df['education_category'] = df['education'].apply(education_group)

In [189]:
df.groupby('education_category')['debt'].agg(['count', 'mean'])

,count,mean
education_category,,
school,16259,0.089981
university,5266,0.052791


Клиенты у кого есть высшее образование возвращают кредит хуже

In [162]:
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

In [194]:
def family_group(x):
  if x == 'женат / замужем' or x == 'гражданский брак':
    return 'in_wedding'
  else:
    return 'no_wedding'

In [195]:
df['family_status_category'] = df['family_status'].apply(family_group)

In [196]:
df.groupby('family_status_category')['debt'].agg(['count', 'mean'])

,count,mean
family_status_category,,
in_wedding,16557,0.079664
no_wedding,4968,0.084944


Клиенты кторые состоят в браке вызвращают кредит хуже

In [165]:
df['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [197]:
def income_group(x):
  if x == 'сотрудник' or x == 'компаньон' or x == 'госслужащий' or x == 'предприниматель':
    return 'have_job'
  else:
    return 'unemployed'

In [198]:
df['income_type_category'] = df['income_type'].apply(income_group)

In [199]:
df.groupby('income_type_category')['debt'].agg(['count', 'mean'])

,count,mean
income_type_category,,
have_job,17665,0.086216
unemployed,3860,0.056477


Безработные клиенты возвращают кредит хуже

In [168]:
df['total_income'] = df['total_income'].fillna(df['total_income'].median()).astype(int)

In [204]:
df.groupby(pd.cut(df['total_income'], [0,70000,90000,100000,120000,150000,200000, np.inf]))['debt'].agg(['count', 'mean'])

,count,mean
total_income,,
"(0.0, 70000.0]",1474,0.068521
"(70000.0, 90000.0]",1874,0.085912
"(90000.0, 100000.0]",1115,0.082511
"(100000.0, 120000.0]",2382,0.082704
"(120000.0, 150000.0]",5496,0.084425
"(150000.0, 200000.0]",4118,0.089364
"(200000.0, inf]",5066,0.070667


Хуже других клиенты с уровнем дохода до 70000

## Общий вывод:
Клиенты у которых есть дети, не имеют высшего образования, которые не состоят в барке, у которых есть работа и доход превышает 70 000 - лучше других возвращают кредит.